In [13]:
import pandas as pd
import networkx as nx
import numpy as np
import math
import community
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from networkx.algorithms.community import greedy_modularity_communities
from sklearn.cluster import KMeans
#from networkx.algorithms.community.quality import modularity
#from networkx.algorithms.community import modularity_max.greedy_modularity_communities
#import networkx.algorithms.community.modularity_max

In [2]:
%%time

#Import and Merge the First DataFrame
df_1 = pd.read_parquet('src_1min_test_2019-01-09_04_2019-01-09_06.parquet', engine='pyarrow')
df_2 = pd.read_parquet('src_1min_test_2019-01-09_06_2019-01-09_08.parquet', engine='pyarrow')
df_3 = pd.read_parquet('src_1min_test_2019-01-09_08_2019-01-09_10.parquet', engine='pyarrow')
df_4 = pd.read_parquet('src_1min_test_2019-01-09_10_2019-01-09_12.parquet', engine='pyarrow')
df_5 = pd.read_parquet('src_1min_test_2019-01-09_12_2019-01-09_14.parquet', engine='pyarrow')
df_6 = pd.read_parquet('src_1min_test_2019-01-09_14_2019-01-09_16.parquet', engine='pyarrow')
df_7 = pd.read_parquet('src_1min_test_2019-01-09_16_2019-01-09_18.parquet', engine='pyarrow')
df_8 = pd.read_parquet('src_1min_test_2019-01-09_18_2019-01-09_20.parquet', engine='pyarrow')
df_9 = pd.read_parquet('src_1min_test_2019-01-09_20_2019-01-09_22.parquet', engine='pyarrow')
df_10 = pd.read_parquet('src_1min_test_2019-01-09_22_2019-01-10_00.parquet', engine='pyarrow')

frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10]
DataFrame1 = pd.concat(frames)

Wall time: 42.8 s


In [3]:
# drop columns
#DataFrame1 = DataFrame1.drop(['ratio_count', 'ratio_card_port', 'ratio_card_dest', 'cdf_card_port', 'cdf_card_proto', 'ratio_card_proto','ratio_sum_bytes_ts','ratio_sum_bytes_tc'], axis=1)
DataFrame1 = DataFrame1[['src_device','window_start','count','card_dest']]

In [4]:
DataFrame1

,src_device,window_start,count,card_dest
0,5ce41a66-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,8,2
1,5cae9aa8-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1
2,5c73ef34-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1
3,5cd24674-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1
4,5ca5d62a-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1
5,5ccd9cf0-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1
6,5ca944e0-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1
7,5ce67df6-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1
8,5ce6cefa-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1
9,5c41bb72-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,4,1


In [ ]:
# to-do: group by device and time, give trend and distributions on them

In [6]:
DataFrame1[DataFrame1['src_device']=='fb244b1a-b6ed-11e8-b91b-0a580a041177'].sort_values(['window_start'])

,src_device,window_start,count,card_dest
3488366,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:00,2,1
3487291,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:02,2,1
3486298,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:04,2,1
3485021,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:06,2,1
3484312,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:08,2,1
3483319,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:10,2,1
3482327,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:12,2,1
3481325,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:14,2,1
3480539,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:16,2,1
3479645,fb244b1a-b6ed-11e8-b91b-0a580a041177,2019-01-09 04:00:18,2,1


In [4]:
# pivot count time series table and replace nan with zero
table1 = DataFrame1.pivot_table(index='src_device', columns='window_start', values='count', aggfunc='mean').fillna(0)


In [5]:
table1

window_start,2019-01-09 04:00:00,2019-01-09 04:00:02,2019-01-09 04:00:04,2019-01-09 04:00:06,2019-01-09 04:00:08,2019-01-09 04:00:10,2019-01-09 04:00:12,2019-01-09 04:00:14,2019-01-09 04:00:16,2019-01-09 04:00:18,...,2019-01-09 23:59:40,2019-01-09 23:59:42,2019-01-09 23:59:44,2019-01-09 23:59:46,2019-01-09 23:59:48,2019-01-09 23:59:50,2019-01-09 23:59:52,2019-01-09 23:59:54,2019-01-09 23:59:56,2019-01-09 23:59:58
src_device,,,,,,,,,,,,,,,,,,,,,
00960a26-dd1c-11e8-a635-0a580a040e90,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00a42b10-a8c4-11e8-b43d-0a580a140c18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0109633a-1409-11e9-9655-0a580a04027f,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
026760de-f973-11e8-bc0e-0a580a040c2e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03516068-b6a1-11e8-be5d-0a580a041540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0501a6c4-0201-11e9-9461-0a580a04043a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
063b3758-1421-11e9-a646-0a580a040b23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07bc7e74-0563-11e9-b507-0a580a040009,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.00,4.00,4.00,4.00,...,4.5,4.5,4.5,4.5,4.5,4.5,3.0,4.5,4.5,3.0
08893022-a8bb-11e8-b3a8-0a580a14195b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
DataFrame1_up = table1.stack().reset_index(name = 'count')

In [12]:
DataFrame1_up

,src_device,window_start,count
0,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:00,0.0
1,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:02,0.0
2,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:04,0.0
3,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:06,0.0
4,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:08,0.0
5,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:10,0.0
6,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:12,0.0
7,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:14,0.0
8,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:16,0.0
9,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:00:18,0.0


In [7]:
DataFrame1_up['normalize'] = 1 - np.exp(-DataFrame1_up['count'])

In [8]:
DataFrame1_up[DataFrame1_up['normalize']!=0]

,src_device,window_start,count,normalize
216,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:12,1.0,0.632121
217,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:14,1.0,0.632121
218,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:16,1.0,0.632121
219,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:18,1.0,0.632121
220,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:20,1.0,0.632121
221,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:22,1.0,0.632121
222,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:24,1.0,0.632121
223,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:26,1.0,0.632121
224,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:28,1.0,0.632121
225,00960a26-dd1c-11e8-a635-0a580a040e90,2019-01-09 04:07:30,1.0,0.632121


In [32]:
#unpivot and rank

In [36]:
DataFrame1_up.describe()

,count,normalize
count,9.389466e+07,9.389466e+07
mean,1.132212e+00,2.225973e-01
std,3.561599e+00,3.918871e-01
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,1.665000e+02,1.000000e+00


In [9]:
table2 = DataFrame1_up.pivot_table(index='src_device', columns='window_start', values='normalize', aggfunc='mean')

In [10]:
table2

window_start,2019-01-09 04:00:00,2019-01-09 04:00:02,2019-01-09 04:00:04,2019-01-09 04:00:06,2019-01-09 04:00:08,2019-01-09 04:00:10,2019-01-09 04:00:12,2019-01-09 04:00:14,2019-01-09 04:00:16,2019-01-09 04:00:18,...,2019-01-09 23:59:40,2019-01-09 23:59:42,2019-01-09 23:59:44,2019-01-09 23:59:46,2019-01-09 23:59:48,2019-01-09 23:59:50,2019-01-09 23:59:52,2019-01-09 23:59:54,2019-01-09 23:59:56,2019-01-09 23:59:58
src_device,,,,,,,,,,,,,,,,,,,,,
00960a26-dd1c-11e8-a635-0a580a040e90,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00a42b10-a8c4-11e8-b43d-0a580a140c18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0109633a-1409-11e9-9655-0a580a04027f,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
026760de-f973-11e8-bc0e-0a580a040c2e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
03516068-b6a1-11e8-be5d-0a580a041540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0501a6c4-0201-11e9-9461-0a580a04043a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
063b3758-1421-11e9-a646-0a580a040b23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
07bc7e74-0563-11e9-b507-0a580a040009,0.981684,0.981684,0.981684,0.981684,0.981684,0.981684,0.981684,0.981684,0.981684,0.981684,...,0.988891,0.988891,0.988891,0.988891,0.988891,0.988891,0.950213,0.988891,0.988891,0.950213
08893022-a8bb-11e8-b3a8-0a580a14195b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
#normalize rank and pivot again

In [15]:
%%time
#clustering by row
mat = table2.values
# Using sklearn
km = sk.cluster.KMeans(n_clusters=5)
km.fit(mat)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [18]:
km.labels_
unique, counts = np.unique(km.labels_, return_counts=True)
dict(zip(unique, counts))


{0: 1638, 1: 592, 2: 57, 3: 55, 4: 502}

In [19]:
km.inertia_ 

3562674.7653598236

In [22]:
#run different k on clustering, find optimal k
evaluations = []
for i in range(10, 70, 10):
    km = km = sk.cluster.KMeans(n_clusters=i,max_iter=30)
    km.fit(mat)
    evaluations.append(km.inertia_)
print(evaluations)

KeyboardInterrupt: 

In [5]:
DataFrame1.describe()

,count,card_dest
count,3.240551e+07,3.240551e+07
mean,5.613831e+00,1.281003e+00
std,8.772151e+00,1.221611e+00
min,1.000000e+00,1.000000e+00
25%,2.000000e+00,1.000000e+00
50%,3.000000e+00,1.000000e+00
75%,6.000000e+00,1.000000e+00
max,2.000000e+02,5.700000e+01


In [ ]:
#normaliize ranks

In [85]:
#create time series data
mat = table2.values
# Using sklearn
km = sk.cluster.KMeans(n_clusters=5)
km.fit(mat)

AttributeError: module 'sklearn' has no attribute 'cluster'

In [ ]:
#define cluster distance function

In [ ]:
#cluster by functions and compare results

In [8]:
%%time

df_11 = pd.read_parquet('src_dst_1min_test_2019-01-09_04_2019-01-09_06.parquet', engine='pyarrow')
df_12 = pd.read_parquet('src_dst_1min_test_2019-01-09_06_2019-01-09_08.parquet', engine='pyarrow')
df_13 = pd.read_parquet('src_dst_1min_test_2019-01-09_08_2019-01-09_10.parquet', engine='pyarrow')
df_14 = pd.read_parquet('src_dst_1min_test_2019-01-09_10_2019-01-09_12.parquet', engine='pyarrow')
df_15 = pd.read_parquet('src_dst_1min_test_2019-01-09_12_2019-01-09_14.parquet', engine='pyarrow')
df_16 = pd.read_parquet('src_dst_1min_test_2019-01-09_14_2019-01-09_16.parquet', engine='pyarrow')
df_17 = pd.read_parquet('src_dst_1min_test_2019-01-09_16_2019-01-09_18.parquet', engine='pyarrow')
df_18 = pd.read_parquet('src_dst_1min_test_2019-01-09_18_2019-01-09_20.parquet', engine='pyarrow')
df_19 = pd.read_parquet('src_dst_1min_test_2019-01-09_20_2019-01-09_22.parquet', engine='pyarrow')
df_20 = pd.read_parquet('src_dst_1min_test_2019-01-09_22_2019-01-10_00.parquet', engine='pyarrow')

frames2 = [df_11, df_12, df_13, df_14, df_15, df_16, df_17, df_18, df_19, df_20]
DataFrame2 = pd.concat(frames2)

Wall time: 53.1 s


In [9]:
DataFrame2

,src_device,dst_device,window_start,count,card_port,card_proto,sum_bytes_ts,sum_bytes_tc,cdf_count,ratio_count,cdf_card_port,ratio_card_port,cdf_card_proto,ratio_card_proto,cdf_sum_bytes_ts,ratio_sum_bytes_ts,cdf_sum_bytes_tc,ratio_sum_bytes_tc
0,5ce67df6-b12f-11e8-88c7-0a580a041c07,3cfff7fa-a654-11e8-a1fa-0a580a141e5e,2019-01-09 05:59:58,2,1,1,1918,0,1.000000,NaN,1.000000,NaN,1.0,NaN,1.000000,1.000520,1.000000,NaN
1,5cd24674-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,252,132,0.509677,0.504511,1.000000,NaN,1.0,NaN,0.533333,0.180747,0.503226,0.033690
2,5d094188-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,288,0,1.000000,1.000500,1.000000,NaN,1.0,NaN,1.000000,1.000281,1.000000,NaN
3,5d232ecc-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1,1,78,66,1.000000,NaN,1.000000,NaN,1.0,NaN,1.000000,NaN,1.000000,NaN
4,5ce6cefa-b12f-11e8-88c7-0a580a041c07,3c999b5e-a654-11e8-be3b-0a580a143367,2019-01-09 05:59:58,1,1,1,78,66,1.000000,NaN,1.000000,NaN,1.0,NaN,1.000000,NaN,1.000000,NaN
5,5d0b5a2c-b12f-11e8-88c7-0a580a041c07,3c4a76a0-a654-11e8-a4ed-0a580a140945,2019-01-09 05:59:58,1,1,1,78,66,1.000000,NaN,1.000000,NaN,1.0,NaN,1.000000,NaN,1.000000,NaN
6,5cae9aa8-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,252,132,0.383893,0.256489,1.000000,NaN,1.0,NaN,0.405138,0.109853,0.368083,0.011931
7,5ca5d62a-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1,1,78,66,1.000000,NaN,1.000000,NaN,1.0,NaN,1.000000,NaN,1.000000,NaN
8,fb8062ec-b6ed-11e8-b91b-0a580a041177,a8303abe-a8b9-11e8-8efd-0a580a14198d,2019-01-09 05:59:58,2,1,1,912,0,0.382552,0.133333,0.000000,NaN,1.0,0.500250,0.430102,0.054668,1.000000,NaN
9,fb41d7fc-b6ed-11e8-b91b-0a580a041177,a8303abe-a8b9-11e8-8efd-0a580a14198d,2019-01-09 05:59:58,2,1,1,912,0,0.354944,0.126867,0.000000,NaN,1.0,0.500250,0.399513,0.062092,1.000000,NaN


In [ ]:
DataFrame2.describe()

In [4]:
DataFrame2.head()

,src_device,dst_device,window_start,count,card_port,card_proto,sum_bytes_ts,sum_bytes_tc,cdf_count,ratio_count,cdf_card_port,ratio_card_port,cdf_card_proto,ratio_card_proto,cdf_sum_bytes_ts,ratio_sum_bytes_ts,cdf_sum_bytes_tc,ratio_sum_bytes_tc
0,5ce67df6-b12f-11e8-88c7-0a580a041c07,3cfff7fa-a654-11e8-a1fa-0a580a141e5e,2019-01-09 05:59:58,2,1,1,1918,0,1.000000,NaN,1.0,NaN,1.0,NaN,1.000000,1.000520,1.000000,NaN
1,5cd24674-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,252,132,0.509677,0.504511,1.0,NaN,1.0,NaN,0.533333,0.180747,0.503226,0.03369
2,5d094188-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,288,0,1.000000,1.000500,1.0,NaN,1.0,NaN,1.000000,1.000281,1.000000,NaN
3,5d232ecc-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1,1,78,66,1.000000,NaN,1.0,NaN,1.0,NaN,1.000000,NaN,1.000000,NaN
4,5ce6cefa-b12f-11e8-88c7-0a580a041c07,3c999b5e-a654-11e8-be3b-0a580a143367,2019-01-09 05:59:58,1,1,1,78,66,1.000000,NaN,1.0,NaN,1.0,NaN,1.000000,NaN,1.000000,NaN


In [17]:
# utility, check if there is any null in certain column
print (DataFrame1.isnull().sum())

src_device          0
window_start        0
count               0
card_dest           0
card_port           0
card_proto          0
sum_bytes_ts        0
sum_bytes_tc        0
cdf_count           0
cdf_card_dest       0
cdf_sum_bytes_ts    0
cdf_sum_bytes_tc    0
dtype: int64


In [10]:
# drop terms to create a new connectivity dataframe used for graph
DataFrame2 = DataFrame2.drop(['cdf_card_port','ratio_card_port','ratio_count','cdf_card_proto','ratio_card_proto','ratio_sum_bytes_ts','ratio_sum_bytes_tc'],axis = 1)
DataFrame2.head()

,src_device,dst_device,window_start,count,card_port,card_proto,sum_bytes_ts,sum_bytes_tc,cdf_count,cdf_sum_bytes_ts,cdf_sum_bytes_tc
0,5ce67df6-b12f-11e8-88c7-0a580a041c07,3cfff7fa-a654-11e8-a1fa-0a580a141e5e,2019-01-09 05:59:58,2,1,1,1918,0,1.000000,1.000000,1.000000
1,5cd24674-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,252,132,0.509677,0.533333,0.503226
2,5d094188-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,288,0,1.000000,1.000000,1.000000
3,5d232ecc-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1,1,78,66,1.000000,1.000000,1.000000
4,5ce6cefa-b12f-11e8-88c7-0a580a041c07,3c999b5e-a654-11e8-be3b-0a580a143367,2019-01-09 05:59:58,1,1,1,78,66,1.000000,1.000000,1.000000


In [40]:

DataFrame2 =DataFrame2.drop(['cdf_count','cdf_sum_bytes_ts','cdf_sum_bytes_tc'],axis = 1)
DataFrame2

,src_device,dst_device,window_start,count,card_port,card_proto,sum_bytes_ts,sum_bytes_tc
0,5ce67df6-b12f-11e8-88c7-0a580a041c07,3cfff7fa-a654-11e8-a1fa-0a580a141e5e,2019-01-09 05:59:58,2,1,1,1918,0
1,5cd24674-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,252,132
2,5d094188-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,288,0
3,5d232ecc-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1,1,78,66
4,5ce6cefa-b12f-11e8-88c7-0a580a041c07,3c999b5e-a654-11e8-be3b-0a580a143367,2019-01-09 05:59:58,1,1,1,78,66
5,5d0b5a2c-b12f-11e8-88c7-0a580a041c07,3c4a76a0-a654-11e8-a4ed-0a580a140945,2019-01-09 05:59:58,1,1,1,78,66
6,5cae9aa8-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,2,1,1,252,132
7,5ca5d62a-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:58,1,1,1,78,66
8,fb8062ec-b6ed-11e8-b91b-0a580a041177,a8303abe-a8b9-11e8-8efd-0a580a14198d,2019-01-09 05:59:58,2,1,1,912,0
9,fb41d7fc-b6ed-11e8-b91b-0a580a041177,a8303abe-a8b9-11e8-8efd-0a580a14198d,2019-01-09 05:59:58,2,1,1,912,0


In [11]:
# select rows with 0 starting seconds to eliminate window overlapping

#DataFrame2.dtypes
#pd.to_timedelta(DataFrame2['window_start'],unit='s')
#DataFrame2['window_start'].attributes()
#DataFrame2['window_start'].dt.second
DataFrame4 = DataFrame2[DataFrame2['window_start'].dt.second==0]


In [5]:
#free DataFrame 2
DataFrame2 = None

In [12]:
DataFrame4

,src_device,dst_device,window_start,count,card_port,card_proto,sum_bytes_ts,sum_bytes_tc,cdf_count,cdf_sum_bytes_ts,cdf_sum_bytes_tc
38267,5c950c5a-b12f-11e8-88c7-0a580a041c07,5cc5748a-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:00,1,1,1,86,89,0.000000,0.151899,0.000000
38268,5d03e774-b12f-11e8-88c7-0a580a041c07,f660c5a6-de1e-11e8-a51c-0a580a040cd7,2019-01-09 05:59:00,18,9,1,1614,0,0.725855,0.697677,0.000000
38269,5d03e774-b12f-11e8-88c7-0a580a041c07,f660c5a6-de1e-11e8-a51c-0a580a040cd7,2019-01-09 05:59:00,14,7,1,1264,0,0.622367,0.600133,0.000000
38270,5cd24674-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:00,2,1,1,252,132,0.509677,0.533333,0.503226
38271,5d094188-b12f-11e8-88c7-0a580a041c07,5ca59cb4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:00,2,1,1,288,0,1.000000,1.000000,1.000000
38272,5d232ecc-b12f-11e8-88c7-0a580a041c07,5d2d5780-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:00,1,1,1,78,66,1.000000,1.000000,1.000000
38273,5ce6cefa-b12f-11e8-88c7-0a580a041c07,3c999b5e-a654-11e8-be3b-0a580a143367,2019-01-09 05:59:00,1,1,1,78,66,1.000000,1.000000,1.000000
38274,5d341714-b12f-11e8-88c7-0a580a041c07,5c3e248a-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:00,4,2,1,194,0,0.454936,0.541196,1.000000
38275,5d069a64-b12f-11e8-88c7-0a580a041c07,5cf651a4-b12f-11e8-88c7-0a580a041c07,2019-01-09 05:59:00,1,1,1,125,0,0.000000,0.000000,1.000000
38276,5d0b5a2c-b12f-11e8-88c7-0a580a041c07,3c4a76a0-a654-11e8-a4ed-0a580a140945,2019-01-09 05:59:00,1,1,1,78,66,1.000000,1.000000,1.000000


In [13]:
#group the data by src, dest and sum up the counts
DataFrame5 = DataFrame4[['src_device','dst_device','count']]
DataFrame5 = DataFrame5.groupby(['src_device','dst_device'],as_index=False).sum()


In [51]:
#DataFrame5[DataFrame5['dst_device'] == '5cfaa7ae-b12f-11e8-88c7-0a580a041c07']
DataFrame5

,src_device,dst_device,count
0,00960a26-dd1c-11e8-a635-0a580a040e90,0eb9e578-b0c6-11e8-bbd8-0a580a041c07,100
1,00a42b10-a8c4-11e8-b43d-0a580a140c18,0fe919f0-b0c6-11e8-bbd8-0a580a041c07,1
2,0109633a-1409-11e9-9655-0a580a04027f,7268c4b2-b937-11e8-a5da-0a580a04186e,12
3,026760de-f973-11e8-bc0e-0a580a040c2e,5d0f5492-b12f-11e8-88c7-0a580a041c07,179
4,03516068-b6a1-11e8-be5d-0a580a041540,5d0e8e36-b12f-11e8-88c7-0a580a041c07,109
5,0501a6c4-0201-11e9-9461-0a580a04043a,5ccaa860-b12f-11e8-88c7-0a580a041c07,11
6,0501a6c4-0201-11e9-9461-0a580a04043a,5cfaa7ae-b12f-11e8-88c7-0a580a041c07,29
7,063b3758-1421-11e9-a646-0a580a040b23,7268c4b2-b937-11e8-a5da-0a580a04186e,1
8,07bc7e74-0563-11e9-b507-0a580a040009,0ff94712-b0c6-11e8-bbd8-0a580a041c07,6706
9,08504a78-0905-11e9-bcf3-0a580a040b15,5d0fc896-b12f-11e8-88c7-0a580a041c07,1


In [54]:
# create multigraph using connecttivity data
#MDG= nx.from_pandas_edgelist(DataFrame5, 'src_device', 'dst_device',create_using=nx.MultiDiGraph())

In [55]:
#MDG

In [31]:
G= nx.from_pandas_edgelist(DataFrame5, 'src_device', 'dst_device',create_using=nx.Graph())

In [29]:
# Create directed graph using df5
DG= nx.from_pandas_edgelist(DataFrame5, 'src_device', 'dst_device',['count'],create_using=nx.DiGraph())

In [26]:
DG

In [ ]:
#finding degree, centrality and others using directed graph

In [32]:
#graph grouping

from networkx.algorithms.community import greedy_modularity_communities
#part = community.best_partition(DG)
c = list(greedy_modularity_communities(G))


frozenset({'00aae5a4-a658-11e8-a1b2-0a580a140b2c',
           '00b83710-cb40-11e8-8abe-0a580a0408fb',
           '01473258-cb40-11e8-82c5-0a580a041012',
           '065c813e-dd34-11e8-acdd-0a580a040c20',
           '09df5f34-a675-11e8-a314-0a580a14096c',
           '10e37c68-b69a-11e8-868b-0a580a041f94',
           '11aacbd4-a66d-11e8-a5a2-0a580a141ae3',
           '1255b604-a661-11e8-bdd5-0a580a14235d',
           '14e1403e-a655-11e8-b1e9-0a580a14095e',
           '14f85134-a655-11e8-b2be-0a580a140e37',
           '183b9cb2-a708-11e8-9564-0a580a142072',
           '18de0eea-a661-11e8-bcce-0a580a141bcf',
           '19113bcc-b42a-11e8-928b-0a580a04060e',
           '19586dd0-b42a-11e8-98cc-0a580a041dd1',
           '1ac5e45e-d3c4-11e8-9f70-0a580a04109f',
           '1d62bebe-f96f-11e8-be0d-0a580a04060f',
           '1ee56092-ee9e-11e8-826f-0a580a04071b',
           '1f14c2a4-f4b3-11e8-8bbc-0a580a04096c',
           '1f7ad760-13df-11e9-a7e0-0a580a040282',
           '284ef626-c8e6-11e8-

frozenset({'00960a26-dd1c-11e8-a635-0a580a040e90',
           '07bc7e74-0563-11e9-b507-0a580a040009',
           '08893022-a8bb-11e8-b3a8-0a580a14195b',
           '0a9c0862-e353-11e8-9ca7-0a580a0414dd',
           '0c0bbb14-0781-11e9-8583-0a580a040252',
           '0d37893a-0909-11e9-a3ba-0a580a040a17',
           '0ea67f2e-b0c6-11e8-bbd8-0a580a041c07',
           '0ea8a15a-b0c6-11e8-bbd8-0a580a041c07',
           '0ea8d62a-b0c6-11e8-bbd8-0a580a041c07',
           '0ea93818-b0c6-11e8-bbd8-0a580a041c07',
           '0ea94f7e-b0c6-11e8-bbd8-0a580a041c07',
           '0ea9549c-b0c6-11e8-bbd8-0a580a041c07',
           '0ea9599c-b0c6-11e8-bbd8-0a580a041c07',
           '0ea96b76-b0c6-11e8-bbd8-0a580a041c07',
           '0ea97012-b0c6-11e8-bbd8-0a580a041c07',
           '0ea9ac3a-b0c6-11e8-bbd8-0a580a041c07',
           '0eaa0130-b0c6-11e8-bbd8-0a580a041c07',
           '0eaa2246-b0c6-11e8-bbd8-0a580a041c07',
           '0eaa2d0e-b0c6-11e8-bbd8-0a580a041c07',
           '0eaa3fd8-b0c6-11e8-